In [5]:
# このセルを実行して環境のチェックをしてください。

!nvidia-smi
# nvidia-smi: not found  になる場合は、ランタイム→ランタイムのタイプを変更→ハードウェアアクセラレータを GPU に変更してください。

import tensorflow as tf

# あとでつかうデータセットをダウンロードしておきます。
tf.keras.datasets.fashion_mnist.load_data()
tf.keras.datasets.mnist.load_data() 
print('OK')

/bin/sh: 1: nvidia-smi: not found
OK



## 機械学習と deep learning のイメージを掴む

典型的な機械学習とは、以下のようなプロセスを繰り返すものです。

```python
f = initialize_function()
for _ in range(1000):
  y = f(x)
  l = loss(y)
  f = update_function_to_minimize_loss(f, l)
```

ゴールは「入力データから（なんらかの基準に基づいて）最適な出力を得られるような関数」を得ることです。

1. 適当な関数を定義する (例えば `f(x) = a * x ^ 2 + b * x + c, a = 1.2, b = 0.3, c = -1.4`)
  - 必ずしも数学的な式で表せる必要はありません。たとえば if 文の塊で表現する場合もあります。（RandomForest）
2. 学習のためのデータをその関数に入れ、何らかの出力を得る (`y = f(x)`)
3. その出力がどのくらい良くなかったかを計算する (= 損失)
  - 「猫の画像」に対して「猫である確率 0.2 」と予測したら損失は大きくなる
4. 損失が小さくなるように、適当に作った関数のパラメータをいじる
  - 例えば `a = 1.1, b = 0.4, c = -1.1` にする
  - ここで微分が登場します（損失を a, b, c で偏微分すると a, b, c をどっちに動かせば損失が小さくなるかわかる）
  

教師あり学習の場合は、関数によって予測された `y` が正解データ `y_true` にどれだけ近いかが損失になります。


## パーセプトロン

deep learning とは深く層を重ねたニューラルネットワークを用いた学習のことです。
ニューラルネットワークとは、たとえば以下のようなものから構成されます。

deep learning のもっとも単純な構成要素は以下のような式でイメージできます。（これがニューラルネットワークの一層だと考えてください。）

```python
f(x) = w * x + b
```

ここで登場する `w, b` を↑で説明したようなステップで最適化することで、良い予測モデルをつくることができます。
一方で、この単純な関数では表現力が弱すぎるため、データが複雑な関係性を持っている場合には良い予測ができなくなります。
（たとえば、真の分布を表現する関数が `x ^ 2` だった場合を考えます。どう `w` や `b` を定義しても `w * x + b` では `x ^ 2` は表せません。）

そこで、もっと層を重ねることで表現力を増すという方法を考えます。
単純にやると以下のようになります。

```python
f(x) = w_2 * (w_1 * x + b_1) + b_2
```

実はこのままでは表現力は増してくれません。
上の式を整理すると

```
f(x) = w'x + b'
(ただし、w' = w_2 * w_1, b' = w_2 * b_1 + b2)
```

と表せてしまいます。つまり `f(x) = w * x + b` と同じ表現力になってしまうことがわかります。

## 活性化関数 / 非線形

そこで登場するのが **活性化関数** です。
簡単に言うと、層と層の間に挟む、ちょっと特殊な関数のことです。（※ 非線形な関数である必要があります。）
活性化関数を `h` という名前で呼ぶとすると、

```
f(x) = w_2 * h(w_1 * x + b_1) + b_2
```

となります。式変形しても `f(x) = w * x + b` と表せなくなれば、一層で表される関数より複雑な表現力を持つ関数を作れたことになります。